In [1]:
import sys
sys.path.append('../')
import matplotlib.pyplot as plt

from Miniproject_2.model import *

import torch
from torch.nn import functional as F

torch.set_grad_enabled(True);

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

valid_input, valid_target = torch.load('../val_data.pkl',map_location=device)#validation set (noise-clean)
train_input, train_target = torch.load('../train_data.pkl',map_location=device) #test set (noise-noise)

num_samples = 1000
valid_input=torch.narrow(valid_input,0,0,num_samples)
valid_target=torch.narrow(valid_target,0,0,num_samples)
train_input=torch.narrow(train_input,0,0,num_samples)
train_target=torch.narrow(train_target,0,0,num_samples)

# plt.imshow(valid_input[164].permute(1,2,0))

print("Vector shape: ",train_input.shape)

Vector shape:  torch.Size([1000, 3, 32, 32])


In [3]:
def compare(x,y,decimals=7):
    return torch.all(torch.round(torch.abs(x - y), decimals=decimals)==0.).item()

## Test MSE loss derivative

In [45]:
class MSELoss(Module):
    def __init__(self, *input):
        super(MSELoss,self).__init__()
        self.input = None
        self.reference = None
    
    def forward(self,input,reference):
        self.input     = input
        self.reference = reference
        n              = input.size().numel()
        output         = ((input-reference)**2).sum()/n
        return output

    __call__ = forward

    def backward(self):
        return 2*(self.input - self.reference)/self.input.size().numel()



In [54]:
# Test MSE loss
idx = 164
len = 1
y=(valid_input[idx].float()/255.).requires_grad_()
y_true=(valid_target[idx].float()/255.).requires_grad_()

# fig,ax = plt.subplots(1,2, figsize=(8,8))
# ax[0].imshow(input.permute(1,2,0))
# ax[1].imshow(target.permute(1,2,0))
L = MSELoss()
out = L.forward(y,y_true)

L_true = F.mse_loss(y,y_true, reduction='mean')
print('Output: ', L_true, '\nTrue output: ', out)


Output:  tensor(0.0110, grad_fn=<MseLossBackward0>) 
True output:  tensor(0.0110, grad_fn=<DivBackward0>)


In [55]:
dL_dy = torch.autograd.grad(L_true,(y))[0]
dLdy = L.backward()

print(compare(dL_dy, dLdy, decimals=3))
print(dLdy.shape)

True
torch.Size([3, 32, 32])


## Test Sigmoid derivative

In [39]:
class Sigmoid(Module):
    def __init__(self,*input):
        super(Sigmoid,self).__init__()
        self.input = None
        return 

    def forward(self,input):
        self.input = input
        output     = 1 / (1 + (-input).exp())
        return output
    
    __call__ = forward

    def backward(self):
        x = self.input
        return (-x).exp()/(1+ (-x).exp())**2

In [43]:
# Test MSE loss
idx = 164
len = 1
y=(valid_input[idx].float()/255.).requires_grad_()
y_true=(valid_target[idx].float()/255.).requires_grad_()

# fig,ax = plt.subplots(1,2, figsize=(8,8))
# ax[0].imshow(input.permute(1,2,0))
# ax[1].imshow(target.permute(1,2,0))
S = Sigmoid()
out = S.forward(y)

S_true = torch.sigmoid(y)
# print('Output: ', S_true, '\nTrue output: ', out)
print(compare(S_true, out, decimals=6))

dSdy = S.backward()

True
tensor([[0.2395, 0.2393, 0.2393,  ..., 0.2377, 0.2364, 0.2385],
        [0.2397, 0.2391, 0.2406,  ..., 0.2379, 0.2377, 0.2371],
        [0.2401, 0.2383, 0.2318,  ..., 0.2375, 0.2373, 0.2375],
        ...,
        [0.2424, 0.2425, 0.2451,  ..., 0.2362, 0.2244, 0.2216],
        [0.2379, 0.2373, 0.2403,  ..., 0.2244, 0.2465, 0.2430],
        [0.2373, 0.2362, 0.2412,  ..., 0.2488, 0.2481, 0.2474]],
       grad_fn=<SelectBackward0>)


In [34]:
S1=Sigmoid()

testS = Sequential(S1)





In [8]:
# plt.imshow(out[0].permute(1,2,0)[:,:,::2])

## Test Sequential

In [9]:
f = torch.empty(5,3,3,3)

f[0, 0] = torch.tensor([ [ +0., +0., -1. ], [ +0., +1., +0. ], [ -1., +0., +0. ]])
f[1, 0] = torch.tensor([ [ +1., +1., +1. ], [ +1., +1., +1. ], [ +1., +1., +1. ]])
f[2, 0] = torch.tensor([ [ -1., +0., +1. ], [ -1., +0., +1. ], [ -1., +0., +1. ]])
f[3, 0] = torch.tensor([ [ -1., -1., -1. ], [ +0., +0., +0. ], [ +1., +1., +1. ]])
f[4, 0] = torch.tensor([ [ +0., -1., +0. ], [ -1., +4., -1. ], [ +0., -1., +0. ]])

for j in range(0,5):
    for i in range(1,3):
        f[j,i] = f[j,0]

ff = f.transpose(0,1)
ff.requires_grad_();

In [10]:
stride = 2
kernel_size = 2

conv1 = Conv2d(3,5, kernel_size, stride=stride, padding=0, dilation=1)
# conv1.weight=f
relu1 = Relu()
conv2 = Conv2d(5,5,kernel_size, stride=stride, padding=0, dilation=1)
relu2 = Relu()
tconv3 = ConvTranspose2d(5,5, kernel_size, stride=stride, padding=0, dilation=1)
relu3 = Relu()
tconv4 = ConvTranspose2d(5,3, kernel_size, stride=stride, padding=0, dilation=1)
sig4 = Sigmoid()

net = Sequential(conv1,  
                relu1,  
                conv2, 
                relu2, 
                tconv3,
                relu3, 
                tconv4,
                sig4
                )

In [11]:
idx = 164
len = 1
a=valid_input[[idx]].float()/255.
target=valid_target[idx].float()/255.
# plt.imshow(a[0].permute(1,2,0))

In [12]:
out=net(a)
# plt.imshow(out[0].permute(1,2,0)[:,:,:])
# plt.colorbar()

In [50]:
out.shape

torch.Size([1, 3, 32, 32])